# Equal-Weight S&P 500 Index Fund

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [4]:
from secrets import IEX_CLOUD_API_TOKEN

In [5]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': ' KACT)DRAG(S/B QLMNASLAEESEL OGTN',
 'calculationPrice': 'tops',
 'open': 0,
 'openTime': None,
 'openSource': 'ifilofac',
 'close': 0,
 'closeTime': None,
 'closeSource': 'ilaiffco',
 'high': 0,
 'highTime': 1631826856507,
 'highSource': 'i pXt lemerIEe iacr',
 'low': 0,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 138.91,
 'latestSource': 'IEX real time price',
 'latestTime': '9:53:34 AM',
 'latestUpdate': 1696611589240,
 'latestVolume': None,
 'iexRealtimePrice': 144.97,
 'iexRealtimeSize': 100,
 'iexLastUpdated': 1633420827589,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 140.67,
 'previousVolume': 52563548,
 'change': 1.48,
 'changePercent': 0.01089,
 'volume': None,
 'iexMarketPercent': 0.02036839778483837,
 'iex

In [6]:
price = data['latestPrice']
marketCap = data['marketCap']

## Using Batch API Calls in order to reduce time required in the process

In [7]:
df_columns = ['Ticker','Stock Price','Market Capitalization','Number of Shares to Buy']
df = pd.DataFrame(columns = df_columns)

In [8]:
def create_batches(x,n):
    for i in range(0,len(x),n):
        yield x[i:i+n]

In [9]:
ticker_batches_list = list(create_batches(stocks['Ticker'],100))

In [10]:
ticker_batches = list()
for i in range(0, len(ticker_batches_list)):
    ticker_batches.append(','.join(ticker_batches_list[i]))

In [11]:
for batch_string in ticker_batches:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={batch_string}&token={IEX_CLOUD_API_TOKEN}'

    data = requests.get(batch_api_url).json()

    for ticker in batch_string.split(','):
        df = df.append(
            pd.Series(
                [
                    ticker, 
                    data[ticker]['quote']['latestPrice'], 
                    data[ticker]['quote']['marketCap'], 
                    'N/A'
                ], index = df_columns
            ), ignore_index = True
        )

In [12]:
df.head()

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,155.080,47058647356,N/A
1,AAL,22.020,13697949500,N/A
2,AAP,213.330,13729929363,N/A
3,AAPL,141.249,2343545365816,N/A
4,ABBV,115.400,205432296870,N/A


## Mapping the Portfolio size to position size and calculating the amount of stocks to be bought

In [13]:
portfolio_size = input(' Enter the value of your portfolio : ')
try:
    value = float(portfolio_size)
except ValueError:
    value = float(input(' Please enter the integer value of your portfolio : '))

In [14]:
position_size = value/len(df)

In [15]:
portfolio_size,position_size

('1456452', 2884.063366336634)

In [16]:
for i in range(0,len(df.index)):
    df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/df['Stock Price'][i])

In [17]:
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,155.080,47058647356,18
1,AAL,22.020,13697949500,130
2,AAP,213.330,13729929363,13
3,AAPL,141.249,2343545365816,20
4,ABBV,115.400,205432296870,24
...,...,...,...,...
500,YUM,117.580,35686928114,24
501,ZBH,169.753,35663188582,16
502,ZBRA,539.068,28693278427,5
503,ZION,55.400,8927101668,52


## Writing the complete data to the XLSX file

In [18]:
xlsxwriter = pd.ExcelWriter('recommended_trades.xlsx',engine='xlsxwriter')
df.to_excel(xlsxwriter,'Recommended Trade',index=False)

In [19]:
bg_color = '#0a0a23'
fg_color = '#ffffff'
string_format = xlsxwriter.book.add_format(
    {
        'font_color':fg_color,
        'bg_color':bg_color,
        'border': 1
    }
)

dollar_format = xlsxwriter.book.add_format(
    {
        'num_format':'$0.00',
        'font_color':fg_color,
        'bg_color':bg_color,
        'border': 1
    }
)

int_format = xlsxwriter.book.add_format(
    {
        'num_format':'0.00',
        'font_color':fg_color,
        'bg_color':bg_color,
        'border': 1
    }
)

In [20]:
xlsxwriter.sheets['Recommended Trade'].set_column('A:A',18,string_format)

0

In [21]:
xlsxwriter.sheets['Recommended Trade'].set_column('B:B',18,dollar_format)

0

In [22]:
xlsxwriter.sheets['Recommended Trade'].set_column('C:C',18,int_format)

0

In [23]:
xlsxwriter.sheets['Recommended Trade'].set_column('D:D',18,int_format)

0

In [24]:
column_format = {
    'A':['Ticker',string_format],
    'B':['Stock Price',dollar_format],
    'C':['Market Capitalization',int_format],
    'D':['Number of Share to Buy',int_format]
}

In [25]:
for column in column_format.keys():
    xlsxwriter.sheets['Recommended Trade'].set_column(f'{column}:{column}', 20, column_format[column][1])
    xlsxwriter.sheets['Recommended Trade'].write(f'{column}1', column_format[column][0], string_format)

In [26]:
xlsxwriter.save()